# Machine Learning Models on the ids 2017

In this notebook, random tree and random forest based machine learning algorithms are applied to the ids2017 dataset. Several methods for resolving the class imbalance are tested. Random tree algorithms were chosen for their effectiveness and the training time which were better than other machine learning models. 

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, average_precision_score, make_scorer, precision_score, accuracy_score, confusion_matrix
from notebook_utils import upsample_dataset
%matplotlib inline
%load_ext autoreload
%autoreload 2
file_path = r"..\CIC-IDS-2017\CSVs\GeneratedLabelledFlows\TrafficLabelling\processed\ids2017_processed.csv"

def load_dataset(file_path):
    df = pd.read_csv(file_path)
    convert_dict = {'label': 'category'}
    df = df.astype(convert_dict)
    df.info()
    return df

attack_labels = {
    0: 'BENIGN',
    7: 'FTP-Patator',
    11: 'SSH-Patator',
    6: 'DoS slowloris',
    5: 'DoS Slowhttptest',
    4: 'DoS Hulk',
    3: 'DoS GoldenEye',
    8: 'Heartbleed',
    12: 'Web Attack - Brute Force',
    14: 'Web Attack - XSS',
    13: 'Web Attack - Sql Injection',
    9: 'Infiltration',
    1: 'Bot',
    10: 'PortScan',
    2: 'DDoS'
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preparing the Dataset

In [8]:
df = load_dataset(file_path)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 96 columns):
 #   Column                       Dtype   
---  ------                       -----   
 0   destination_port             int64   
 1   protocol                     int64   
 2   flow_duration                int64   
 3   total_fwd_packets            int64   
 4   total_backward_packets       int64   
 5   total_length_of_fwd_packets  float64 
 6   total_length_of_bwd_packets  float64 
 7   fwd_packet_length_max        float64 
 8   fwd_packet_length_min        float64 
 9   fwd_packet_length_mean       float64 
 10  fwd_packet_length_std        float64 
 11  bwd_packet_length_max        float64 
 12  bwd_packet_length_min        float64 
 13  bwd_packet_length_mean       float64 
 14  bwd_packet_length_std        float64 
 15  flow_bytes_s                 float64 
 16  flow_packets_s               float64 
 17  flow_iat_mean                float64 
 18  flow_iat_std          

#### Check for invalid values

In [12]:
# Select only numeric columns
numeric_columns = df.select_dtypes(include=[np.number]).columns

# Identify columns with NaN, infinity, or negative values
nan_columns = df[numeric_columns].columns[df[numeric_columns].isna().any()]
inf_columns = df[numeric_columns].columns[np.isinf(df[numeric_columns]).any()]
neg_columns = df[numeric_columns].columns[(df[numeric_columns] < 0).any()]

print("Columns with NaN values:", nan_columns.tolist())
print("Columns with infinite values:", inf_columns.tolist())
print("Columns with negative values:", neg_columns.tolist())

# Calculate the percentage of NaN, infinite, and negative values
nan_percentage = df[nan_columns].isna().mean() * 100
inf_percentage = df[inf_columns].map(lambda x: np.isinf(x)).mean() * 100
neg_percentage = df[neg_columns].map(lambda x: x < 0).mean() * 100

print("Percentage of NaN values in each column:\n", nan_percentage)
print("Percentage of infinite values in each column:\n", inf_percentage)
print("Percentage of negative values in each column:\n", neg_percentage)


Columns with NaN values: ['flow_bytes_s']
Columns with infinite values: ['flow_bytes_s', 'flow_packets_s']
Columns with negative values: ['flow_duration', 'flow_bytes_s', 'flow_packets_s', 'flow_iat_mean', 'flow_iat_max', 'flow_iat_min', 'fwd_iat_min', 'fwd_header_length', 'bwd_header_length', 'fwd_header_length_1', 'init_win_bytes_forward', 'init_win_bytes_backward', 'min_seg_size_forward']
Percentage of NaN values in each column:
 flow_bytes_s    0.047973
dtype: float64
Percentage of infinite values in each column:
 flow_bytes_s      0.053308
flow_packets_s    0.101281
dtype: float64
Percentage of negative values in each column:
 flow_duration               0.004063
flow_bytes_s                0.003003
flow_packets_s              0.004063
flow_iat_mean               0.004063
flow_iat_max                0.004063
flow_iat_min                0.102129
fwd_iat_min                 0.000601
fwd_header_length           0.001236
bwd_header_length           0.000777
fwd_header_length_1        

Given the low percentage of null values in only one column (0.4%), it is safe to drop the rows with NaN values. The same applies to th infinite values. For negative values, 2 columns have an extremely high percentage of negative values. We choose to drop the features "init_win_bytes_forward" and "init_win_bytes_backward" as the source of the negative sign is unknown. For the rest of the features, the percentages are low so the rows with negative values are dropped.

In [13]:
def replace_invalid(df):
    # Select only numeric columns
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    
    # Identify columns with NaN, infinite, or negative values
    nan_columns = df[numeric_columns].columns[df[numeric_columns].isna().any()]
    inf_columns = df[numeric_columns].columns[np.isinf(df[numeric_columns]).any()]
    neg_columns = df[numeric_columns].columns[(df[numeric_columns] < 0).any()]

    # Drop rows with NaN values (low percentage of NaN values)
    df = df.dropna(subset=nan_columns)

    # Drop rows with infinite values (assuming low percentage)
    for col in inf_columns:
        df = df[np.isfinite(df[col])]

    # Drop columns with a high percentage of negative values
    columns_to_drop = ['init_win_bytes_forward', 'init_win_bytes_backward']
    df = df.drop(columns=columns_to_drop)

    # Drop rows with negative values in the remaining columns
    remaining_neg_columns = [col for col in neg_columns if col not in columns_to_drop]
    for col in remaining_neg_columns:
        df = df[df[col] >= 0]
    
    return df

In [14]:
df = replace_invalid(df)

## Feature Selection